# Generate informations for normalization

In [ ]:
import numpy as np
import os

def compute_train_stats(folder_path):
    """Loads all .npy files in the folder and calculates the mean and standard deviation."""
    all_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".npy"):
            file_path = os.path.join(folder_path, filename)
            data = np.load(file_path)

            if data.ndim == 1:  
                data = data.reshape(1, -1)

            all_data.append(data)

    if not all_data:
        raise ValueError("No .npy files found in the folder!")

    all_data = np.vstack(all_data)

    train_mean = np.mean(all_data, axis=0)
    train_std = np.std(all_data, axis=0)

    # Speicherort neben dem "keypoints" Ordner
    parent_folder = os.path.dirname(folder_path)

    np.save(os.path.join(parent_folder, "train_mean.npy"), train_mean)
    np.save(os.path.join(parent_folder, "train_std.npy"), train_std)

    print("✅ Mean & standard deviation saved next to the keypoints folder!")

# Example call:
compute_train_stats("/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/keypoints")

# Training

In [35]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

# Paths
LABELS_PATH = "/home/haggenmueller/asl_detection/machine_learning/models/lstm/label_to_index.json"
DATA_DIR = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/normalized_keypoints"
MODEL_PATH = "/home/haggenmueller/asl_detection/machine_learning/models/lstm/best_lstm_model.pth"

# Parameters
SEQUENCE_LENGTH = 102
INPUT_SIZE = 225
BATCH_SIZE = 64
EPOCHS = 200  
LR = 0.001  
PATIENCE = 3

# Load labels
with open(LABELS_PATH, "r") as f:
    label_to_index = json.load(f)
    index_to_label = {v: k for k, v in label_to_index.items()}
    NUM_CLASSES = len(label_to_index)

# Dataset class
class ASLDataset(Dataset):
    def __init__(self, samples, sequence_length):
        self.samples = samples
        self.sequence_length = sequence_length
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        keypoints = np.load(file_path)
        
        if keypoints.shape[0] < self.sequence_length:
            pad = np.zeros((self.sequence_length - keypoints.shape[0], keypoints.shape[1]))
            keypoints = np.vstack((keypoints, pad))
        else:
            keypoints = keypoints[:self.sequence_length]
        
        # Normalize keypoints
        keypoints = (keypoints - keypoints.mean()) / keypoints.std()
        
        return torch.tensor(keypoints, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Load and split dataset
label_samples = defaultdict(list)
for file in os.listdir(DATA_DIR):
    if file.endswith(".npy"):
        filename_parts = file.split("_")
        label_name = filename_parts[1]
        if label_name in label_to_index:
            label_samples[label_name].append(os.path.join(DATA_DIR, file))

train_samples, val_samples, test_samples = [], [], []

for label, files in label_samples.items():
    np.random.shuffle(files)
    num_total = len(files)
    
    num_train = int(0.70 * num_total)
    num_val = int(0.15 * num_total)
    num_test = num_total - num_train - num_val

    train_samples.extend([(f, label_to_index[label]) for f in files[:num_train]])
    val_samples.extend([(f, label_to_index[label]) for f in files[num_train:num_train + num_val]])
    test_samples.extend([(f, label_to_index[label]) for f in files[num_train + num_val:]])

train_dataset = ASLDataset(train_samples, SEQUENCE_LENGTH)
val_dataset = ASLDataset(val_samples, SEQUENCE_LENGTH)
test_dataset = ASLDataset(test_samples, SEQUENCE_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.0)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])  # Extract last time step
        return self.fc(out)

# Initialize model
model = LSTMModel(input_size=INPUT_SIZE, hidden_size=256, num_layers=2, num_classes=NUM_CLASSES)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.1)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if "weight_ih" in name or "weight_hh" in name:
                nn.init.xavier_uniform_(param)
            elif "bias" in name:
                nn.init.constant_(param, 0.1)

model.apply(init_weights)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=0.3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

# Training loop
best_loss = float("inf")
stopping_counter = 0
for epoch in range(EPOCHS):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    for keypoints, labels in train_loader:
        keypoints, labels = keypoints.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(keypoints)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()
        
        _, predicted = torch.max(torch.softmax(outputs, dim=1), 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    scheduler.step()
    
    if avg_loss < best_loss:
        best_loss = avg_loss
        stopping_counter = 0
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"✅ Best model saved at epoch {epoch+1} with loss {avg_loss:.4f}")
    else:
        stopping_counter += 1
        if stopping_counter >= PATIENCE:
            print("Early stopping triggered.")
            break

print("Training completed! Best model saved at:", MODEL_PATH)

# Testing the model
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

def evaluate(loader, name):
    correct, total, loss = 0, 0, 0
    with torch.no_grad():
        for keypoints, labels in loader:
            keypoints, labels = keypoints.to(device), labels.to(device)
            outputs = model(keypoints)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(torch.softmax(outputs, dim=1), 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    accuracy = 100 * correct / total
    print(f"{name} Set: Loss = {loss / len(loader):.4f}, Accuracy = {accuracy:.2f}%")

evaluate(val_loader, "Validation")
evaluate(test_loader, "Test")

Epoch [1/200], Loss: 5.0444, Accuracy: 1.69%
✅ Best model saved at epoch 1 with loss 5.0444
Epoch [2/200], Loss: 4.6239, Accuracy: 4.24%
✅ Best model saved at epoch 2 with loss 4.6239
Epoch [3/200], Loss: 4.4529, Accuracy: 5.93%
✅ Best model saved at epoch 3 with loss 4.4529
Epoch [4/200], Loss: 4.2901, Accuracy: 7.78%
✅ Best model saved at epoch 4 with loss 4.2901
Epoch [5/200], Loss: 4.1312, Accuracy: 10.03%
✅ Best model saved at epoch 5 with loss 4.1312
Epoch [6/200], Loss: 3.9891, Accuracy: 11.74%
✅ Best model saved at epoch 6 with loss 3.9891
Epoch [7/200], Loss: 3.8821, Accuracy: 13.65%
✅ Best model saved at epoch 7 with loss 3.8821
Epoch [8/200], Loss: 3.7913, Accuracy: 15.02%
✅ Best model saved at epoch 8 with loss 3.7913
Epoch [9/200], Loss: 3.7178, Accuracy: 18.10%
✅ Best model saved at epoch 9 with loss 3.7178
Epoch [10/200], Loss: 3.6182, Accuracy: 19.21%
✅ Best model saved at epoch 10 with loss 3.6182
Epoch [11/200], Loss: 3.5275, Accuracy: 21.57%
✅ Best model saved at epo

/tmp/ipykernel_3023638/1084232635.py:161: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))


Validation Set: Loss = 1.6493, Accuracy = 79.59%
Test Set: Loss = 1.6276, Accuracy = 81.02%
